- Họ và tên: Nguyễn Khánh Nghĩa
- Lớp KTDL10
- MSV 7103101336

# WINE CLUSTERING (Phân Cụm Rượu)

# 0. Khởi tạo dữ liệu

In [7]:
import findspark
findspark.init()

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Spark report').getOrCreate()
spark

# 1. Giới thiệu về dữ liệu
Nguồn dữ liệu: https://archive.ics.uci.edu/ml/datasets/wine, https://www.kaggle.com/harrywang/wine-dataset-for-clustering/version/1
Mô tả tập dữ liệu bằng các nguồn thông tin trang chủ cung cấp
Data Set Information:

Những dữ liệu này là kết quả phân tích hóa học của các loại rượu vang được trồng ở cùng một vùng ở Ý nhưng có nguồn gốc từ ba giống rượu khác nhau. Phân tích xác định số lượng của 13 thành phần được tìm thấy trong mỗi loại trong số ba loại rượu.

The attributes are: 
1. Alcohol
2. Malic acid
3. Ash
4. Alcalinity of ash
5. Magnesium
6. Total phenols
7. Flavanoids
8. Nonflavanoid phenols
9. Proanthocyanins
10.Color intensity
11.Hue
12.OD280/OD315 of diluted wines
13.Proline

In a classification context, this is a well posed problem with "well behaved" class structures. A good data set for first testing of a new classifier, but not very challenging.


Attribute Information:

All attributes are continuous

No statistics available, but suggest to standardise variables for certain uses (e.g. for us with classifiers which are NOT scale invariant)

NOTE: 1st attribute is class identifier (1-3)

Bài toán đặt ra: <b> Phân cụm thành phần rượu <b>

<img src="https://miro.medium.com/max/795/1*8BOjsbvSxzbq5zW1pcVLew.png"/> 
<i>

Thông tin về thành phần rượu: 

Malic Acid = một trong những axit chính được tìm thấy trong độ chua của nho. Nồng độ của nó càng giảm khi nho càng chín. Axit malic cung cấp một liên kết chặt chẽ với rượu vang có vị "phẳng" nếu không có đủ. Nếu có quá nhiều rượu sẽ có vị ‘chua’. Điều quan trọng là mức độ axit malic được theo dõi trong quá trình lên men.

Ash = Xác định tro là một phân tích bắt buộc đối với các loại rượu được chứng nhận để đưa ra thị trường. Nó được định nghĩa là tất cả các chất vô cơ còn lại sau khi đốt cháy phần còn lại từ quá trình bay hơi của rượu phải hoặc rượu

Ash_Alcanity = Độ kiềm của tro được định nghĩa là tổng các cation, trừ ion amoni, kết hợp với các axit hữu cơ trong rượu

Magnesium = Các cation magiê có trong rượu vang do sự hấp thụ chất dinh dưỡng của đất trong vườn nho.

Total_Phenols =Hàm lượng phenolic trong rượu dùng để chỉ các hợp chất phenolic — phenol tự nhiên và polyphenol — trong rượu, bao gồm một nhóm lớn gồm hàng trăm hợp chất hóa học ảnh hưởng đến hương vị, màu sắc và cảm giác ngon miệng của rượu.

Flavonoids = Flavonoid là polyphenol chính có trong rượu vang đỏ tính theo trọng lượng, và chiếm khoảng 80 đến 90% tổng số polyphenol

Nonflavanoid_Phenols bao gồm một số phân lớp quan trọng đối với rượu vang, đặc biệt là hydroxycinnamates, stilbene và axit benzoic. 

Proanthocyanins = là polyphenol chính trong rượu vang đỏ đang được nghiên cứu để đánh giá nguy cơ bệnh tim mạch vành và giảm tỷ lệ tử vong nói chung. Với tannin, chúng cũng ảnh hưởng đến hương thơm, hương vị, cảm giác miệng và làm se của rượu vang đỏ..

Color_Intensity = Một phép đo đơn giản về độ tối của rượu bằng cách sử dụng tổng hợp các phép đo độ hấp thụ trong các vùng màu tím, xanh lục và đỏ của quang phổ khả kiến

Proline = Proline thường là axit amin dồi dào nhất có trong nước ép nho và rượu vang.

# 2. Khám Phá dữ liệu (EDA)
<b>Mục đích của EDA</b>
Bước EDA này giúp chúng ta có cái nhìn đầu tiên về dữ liệu. Bạn cần có một cảm giác nhất định về những gì mình có trong tay trước khi có những chiến lược xây dựng mô hình. EDA giúp bạn mường tượng được độ phức tạp của bài toán và vạch ra những bước đầu tiên cần làm.

Việc khám phá dữ liệu không chỉ dừng lại ở lần đầu tiên trước khi xây dựng đặc trưng mà còn cần được thực hiện trong suốt quá trình phát triển hệ thống. Sau khi xây dựng xong các đặc trưng, bạn cũng cần làm lại EDA một lần nữa để xem dữ liệu đã qua xử lý đó đã thực sự sạch chưa. Ngoài ra, sau khi xây dựng và phân tích mô hình, ta cũng thường xuyên cần quay lại EDA để tiếp tục khám phá những điều còn ẩn giấu trong dữ liệu bài toán. Càng hiểu sâu về dữ liệu, bạn sẽ càng sớm giải thích được những hành vi của mô hình và đưa ra những thay đổi phù hợp.

Note

Để phân biệt dữ liệu trước và sau bước tiền xử lý, các cột trong bảng dữ liệu ban đầu được gọi là trường thông tin, trường dữ liệu hoặc đôi khi ngắn gọn là trường. Các cột đã được xử lý và sẵn sàng cho việc huấn luyện mô hình được gọi là đặc trưng.

Vậy, làm EDA là làm những gì?

Kích thước dữ liệu
Trước tiên, bạn cần hình dung được dữ liệu có khoảng bao nhiêu mẫu và có bao nhiêu trường dữ liệu. Nếu dữ liệu quá ít, khả năng cao là bạn không thể dùng Deep Learning để giải quyết mà cần dùng các phương pháp khác. Biết về kích thước dữ liệu cũng giúp bạn xác định kích thước tập huấn luyện (training data) và tập kiểm định (validation data) cũng như chuẩn bị bộ nhớ phù hợp.

Ý nghĩa của từng trường dữ liệu
Làm việc với dữ liệu dạng bảng, bạn cần chuẩn bị tâm lý là sẽ làm việc với dữ liệu nhiều hơn là với mô hình. Biết ý nghĩa của từng trường dữ liệu giúp bạn có những cách xử lý và tạo đặc trưng phù hợp. Ý nghĩa của mỗi trường dữ liệu thường được đi kèm theo bộ dữ liệu hoặc đôi khi bạn phải tự suy luận ra dựa trên các giá trị trong cột. Chẳng hạn, nếu các giá trị trong một cột là “Thái Bình”, “Nghệ An”, “Bến Tre”, … thì khả năng cao cột này là tên tỉnh thành.

Tuy nhiên, có những trường hợp vì lý do bảo mật thông tin mà các giá trị trong mỗi trường đã được mã hóa thành những giá trị vô nghĩa. Với những trường hợp này, việc biết ý nghĩa của trường thông tin là cực kỳ quan trọng. Nó ảnh hưởng trực tiếp tới cách xử lý đặc trưng và chất lượng của mô hình. Nếu bạn không biết ý nghĩa và đặc biệt là dữ liệu được mã hóa dưới dạng số, chúc may mắn vì khả năng cao sẽ đến lúc mô hình có chất lượng không như ý muốn mà bạn không giải thích được.

Kiểu dữ liệu của mỗi trường
Các mô hình Machine Learning cho dữ liệu dạng bảng khá nhạy cảm với kiểu dữ liệu. Nhìn chung, các mô hình Machine Learning đều nhận dữ liệu đã qua xử lý ở dạng số. Một trong những yêu cầu của một mô hình tốt là tính ổn định với những thay đổi nhỏ ở đầu vào. Điều này tức là nếu đầu vào mô hình là những giá trị gần nhau thì đầu ra cũng được kỳ vọng là có giá trị gần nhau. Nếu một dữ liệu dạng hạng mục được mã hóa về số, chẳng hạn mã số người dùng, mà bạn nghĩ nó là dạng số thì mô hình sẽ học được rằng những người dùng có mã số gần nhau sẽ có những đặc trưng gần giống nhau. Điều này khả năng rất cao không đúng, nhất là khi mã số được đánh một cách ngẫu nhiên.

Phân phối xác suất của từng trường
Ta cần nắm được phân phối xác suất của từng trường dữ liệu để lên kế hoạch làm sạch dữ liệu và tạo các đặc trưng liên quan tới dữ liệu đó. Với mỗi cột dữ liệu, những trường hợp sau đây ta cần lưu tâm:

Mọi giá trị trong cột bằng nhau: Ví dụ, trong dữ liệu có một cột là “Năm” và mọi giá trị đều bằng 2020. Như vậy cột này không mang lại ý nghĩa dự đoán. Ta có thể xóa cột này khi làm sạch dữ liệu.

Có quá nhiều giá trị bị khuyết: Nếu thấy ý nghĩa cột này không quan trọng, ta có thể xóa. Nếu nó quan trọng, bạn cần có những chiến lược phù hợp (Xem Xử lý dữ liệu bị khuyết).

Xuất hiện giá trị không hợp lệ: Nếu trường dữ liệu “Tuổi” mang những giá trị là số âm hoặc lớn hơn 200, khả năng cao chúng là những giá trị không hợp lệ. Với những giá trị không hợp lệ, ta có thể gán lại nó về giá trị hợp lệ gần nhất hoặc coi như dữ liệu bị khuyết.

Xuất hiện giá trị ngoại lệ: Giả sử trường thông tin thu nhập hàng tháng chứa hầu hết giá trị trong khoảng từ 1-100 triệu đồng nhưng có một vài trường hợp ngoại lệ kiếm được tới 10 tỉ. Nếu giữ nguyên giá trị 10 tỉ đó, mô hình dường như được huấn luyện gượng ép, nó phải căng sức dự đoán những giá trị ngoại lệ đó khiến chất lượng đối với những giá trị phổ biến bị ảnh hưởng. Ngoài ra, nếu phải làm bước chuẩn hóa dữ liệu về đoạn [0,1] (một kỹ thuật rất phố biến khi xử lý dữ liệu) thì hầu hết các giá trị nằm trong khoảng [0,0.01]. Những giá trị rất nhỏ và gần nhau này có thể dẫn đến việc mô hình không phân biệt được sự khác nhau giữa các mức thu nhập khác nhau. Đó là ví dụ với dữ liệu dạng số. Với dữ liệu dạng hạng mục có nhiều hạng mục khác nhau, nếu một số hạng mục chiếm tới 99% tổng số mẫu trong khi tổng số mẫu của nhiều hạng mục khác lại chỉ có 1%. Ta cần có những cách xử lý đặc biệt với dữ liệu loại này.

Mối tương quan giữa các trường dữ liệu¶
Khi làm EDA, ta cũng cần tính toán độ tương quan giữa các trường dữ liệu, đặc biệt là giữa nhãn dự đoán và các trường còn lại. Nếu hệ số tương quan giữa một cột và cột nhãn bằng không, khả năng cao cột đó không mang lại nhiều giá trị dự đoán. Bạn có thể giành sự ưu tiên cho những cột có độ tương quan lớn hơn. Ngược lại, nếu hệ số tương quan giữa một cột và cột nhãn quá cao, có hai khả năng xảy ra:

Cột đó có khả năng mang lại kết quả dự đoán tốt. Khi đó ta cần tập trung làm sạch và xây dựng các đặc trưng liên quan đến cột này trước.

Dữ liệu có thể bị rò rỉ (data leakage). Giả sử bạn cần dự đoán tuổi của người dùng và nhận ra một cột có hệ số tương quan bằng 1. Nếu đó là cột “Năm sinh” thì rõ ràng bạn chẳng cần làm mô hình Machine Learning mà chỉ cần một phép trừ. Tuy nhiên, bài toán có thể là dự đoán tuổi khi trường “Năm sinh” bị khuyết nhưng biết nhiều thông tin khác. Rõ ràng ta không thể dùng trường “Năm sinh” trong trường hợp này mà cần loại bỏ nó đi.

Ngoài ra, nếu hai cột không phải cột nhãn mà có độ tương quan cao, ta cũng nên kiểm tra ý nghĩa của chúng xem có thể bỏ qua một cột hay không.

Trả lời được những câu hỏi này sẽ giúp ích rất nhiều cho việc làm sạch dữ liệu và xây dựng đặc trưng sau này.

Tùy vào lượng thời gian bạn có và kiến thức của bạn về dữ liệu, bạn có thể phân tích sâu hơn về dữ liệu. Càng hiểu dữ liệu để xây dựng những đặc trưng phù hợp, mô hình của bạn càng có chất lượng tốt hơn.

Tip

Bạn không nên cố gắng dành thật nhiều thời gian vào EDA khi bắt đầu dự án mà chỉ nên dừng lại ở những đánh giá cơ bản trên đây để tìm ra những trường dữ liệu có khả năng cao mang lại kết quả tốt và xây dựng đặc trưng dựa trên những trường đó.

Bạn sẽ còn phải quay lại EDA nhiều lần nữa sau khi xây dựng được mô hình đầu tiên. Chúng ta cần nhanh chóng xây dựng một pipeline hoàn thiện cho bài toán bao gồm xử lý dữ liệu, huấn luyện mô hình và đánh giá chất lượng mô hình. Bạn không cần quá chú trọng vào việc xây dựng một mô hình tốt ngay từ đầu mà nên quan tâm hơn tới một hệ thống hoàn chỉnh để đánh giá chất lượng mô hình và tìm ra những điểm cần cải thiện. Dựa trên những đánh giá đó, bạn có thể đưa ra những suy luận và kiểm chứng chúng bằng dữ liệu. Từ đó đưa ra những điều chỉnh phù hợp.

EDA là công việc tương đối nhàm chán nếu bạn chỉ thích huấn luyện mô hình. Rất may, hiện có rất nhiều thư viện hỗ trợ những chức năng cơ bản được đề cập trên đây. Pandas profiling là một ví dụ.

In [9]:
df = spark.read.csv('wine-clustering (2).csv', header = True)
df.printSchema()

root
 |-- Alcohol: string (nullable = true)
 |-- Malic_Acid: string (nullable = true)
 |-- Ash: string (nullable = true)
 |-- Ash_Alcanity: string (nullable = true)
 |-- Magnesium: string (nullable = true)
 |-- Total_Phenols: string (nullable = true)
 |-- Flavanoids: string (nullable = true)
 |-- Nonflavanoid_Phenols: string (nullable = true)
 |-- Proanthocyanins: string (nullable = true)
 |-- Color_Intensity: string (nullable = true)
 |-- Hue: string (nullable = true)
 |-- OD280: string (nullable = true)
 |-- Proline: string (nullable = true)



In [10]:
#Thay đổi kiểu dữ liệu
df.createOrReplaceTempView("CastData")
df = spark.sql("SELECT FLOAT(alcohol),FLOAT(Malic_Acid),FLOAT(Ash), FLOAT(Ash_Alcanity), FLOAT(Magnesium), FLOAT(Total_Phenols), FLOAT(Flavanoids), FLOAT(Nonflavanoid_Phenols), FLOAT(Proanthocyanins), FLOAT(Color_Intensity), FLOAT(Hue), FLOAT(OD280), FLOAT(Proline) from CastData")
df.printSchema()

root
 |-- alcohol: float (nullable = true)
 |-- Malic_Acid: float (nullable = true)
 |-- Ash: float (nullable = true)
 |-- Ash_Alcanity: float (nullable = true)
 |-- Magnesium: float (nullable = true)
 |-- Total_Phenols: float (nullable = true)
 |-- Flavanoids: float (nullable = true)
 |-- Nonflavanoid_Phenols: float (nullable = true)
 |-- Proanthocyanins: float (nullable = true)
 |-- Color_Intensity: float (nullable = true)
 |-- Hue: float (nullable = true)
 |-- OD280: float (nullable = true)
 |-- Proline: float (nullable = true)



In [11]:
print('Kích thước tập dữ liệu {} dòng, {} cột'.format(df.count(), len(df.columns)))

Kích thước tập dữ liệu 178 dòng, 13 cột


### Mô tả tập dữ liệu

In [12]:
df.limit(10).toPandas()

,alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline
0,14.23,1.71,2.43,15.600000,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.200000,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.600000,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.799999,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.000000,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
5,14.20,1.76,2.45,15.200000,112.0,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450.0
6,14.39,1.87,2.45,14.600000,96.0,2.50,2.52,0.30,1.98,5.25,1.02,3.58,1290.0
7,14.06,2.15,2.61,17.600000,121.0,2.60,2.51,0.31,1.25,5.05,1.06,3.58,1295.0
8,14.83,1.64,2.17,14.000000,97.0,2.80,2.98,0.29,1.98,5.20,1.08,2.85,1045.0
9,13.86,1.35,2.27,16.000000,98.0,2.98,3.15,0.22,1.85,7.22,1.01,3.55,1045.0


In [13]:
df.limit(10).toPandas().transpose()

,0,1,2,3,4,5,6,7,8,9
alcohol,14.23,13.20,13.16,14.370000,13.24,14.20,14.39,14.06,14.83,13.86
Malic_Acid,1.71,1.78,2.36,1.950000,2.59,1.76,1.87,2.15,1.64,1.35
Ash,2.43,2.14,2.67,2.500000,2.87,2.45,2.45,2.61,2.17,2.27
Ash_Alcanity,15.60,11.20,18.60,16.799999,21.00,15.20,14.60,17.60,14.00,16.00
Magnesium,127.00,100.00,101.00,113.000000,118.00,112.00,96.00,121.00,97.00,98.00
Total_Phenols,2.80,2.65,2.80,3.850000,2.80,3.27,2.50,2.60,2.80,2.98
Flavanoids,3.06,2.76,3.24,3.490000,2.69,3.39,2.52,2.51,2.98,3.15
Nonflavanoid_Phenols,0.28,0.26,0.30,0.240000,0.39,0.34,0.30,0.31,0.29,0.22
Proanthocyanins,2.29,1.28,2.81,2.180000,1.82,1.97,1.98,1.25,1.98,1.85
Color_Intensity,5.64,4.38,5.68,7.800000,4.32,6.75,5.25,5.05,5.20,7.22


In [14]:
numeric_cols = [t[0] for t in df.dtypes if t[1] == 'float']
df2 = df.select(numeric_cols).describe().toPandas().transpose()
df2

,0,1,2,3,4
summary,count,mean,stddev,min,max
alcohol,178,13.00061797559931,0.8118265466246918,11.03,14.83
Malic_Acid,178,2.336348312289527,1.1171461004519325,0.74,5.8
Ash,178,2.3665168533164462,0.2743440072687038,1.36,3.23
Ash_Alcanity,178,19.494943806294646,3.339563774928806,10.6,30.0
Magnesium,178,99.74157303370787,14.282483515295652,70.0,162.0
Total_Phenols,178,2.2951123567109697,0.6258510502984755,0.98,3.88
Flavanoids,178,2.0292696758602444,0.9988586941936791,0.34,5.08
Nonflavanoid_Phenols,178,0.3618539305550329,0.12445333944254397,0.13,0.66
Proanthocyanins,178,1.5908988746029606,0.5723588647599089,0.41,3.58


Mô tả dữ liệu của các cột số:
- alcohol: Không bị thiếu giá trị, giá trị trung bình ~13, độ lệch chuẩn: ~0.81, Giá trị nhỏ nhất: 11.03,giá trị lớn nhất: 14.83 
- Malic_Acid: Không bị thiếu giá trị, giá trị trung bình ~2.33, độ lệch chuẩn: ~1.12, Giá trị nhỏ nhất: 0.74,giá trị lớn nhất: 5.8 
- Ash	: Không bị thiếu giá trị, giá trị trung bình ~2.37, độ lệch chuẩn: ~0.27, Giá trị nhỏ nhất:1.36 ,giá trị lớn nhất: 3.23 
- Ash_Alcanity: Không bị thiếu giá trị, giá trị trung bình ~19.5, độ lệch chuẩn: 3.34, Giá trị nhỏ nhất: 10.6 ,giá trị lớn nhất: 30
- Magnesium: Không bị thiếu giá trị, giá trị trung bình ~99.74, độ lệch chuẩn: ~14.28, Giá trị nhỏ nhất: 70,giá trị lớn nhất: 162
- Total_Phenols	: Không bị thiếu giá trị, giá trị trung bình ~2.3, độ lệch chuẩn: ~0.63, Giá trị nhỏ nhất: 0.98,giá trị lớn nhất: 3.88
- Flavanoids: Không bị thiếu giá trị, giá trị trung bình ~2.03, độ lệch chuẩn: ~0.9988, Giá trị nhỏ nhất: 0.34,giá trị lớn nhất: 5.08 
- Nonflavanoid_Phenols: Không bị thiếu giá trị, giá trị trung bình ~0.36, độ lệch chuẩn: ~0.124, Giá trị nhỏ nhất: 0.13,giá trị lớn nhất: 0.66
- Proanthocyanins: Không bị thiếu giá trị, giá trị trung bình ~1.59, độ lệch chuẩn: ~0.57, Giá trị nhỏ nhất: 0.41,giá trị lớn nhất: 3.58 
- Color_Intensity: Không bị thiếu giá trị, giá trị trung bình ~5.06, độ lệch chuẩn: ~2.32, Giá trị nhỏ nhất: 1.28,giá trị lớn nhất: 13 
- Hue: Không bị thiếu giá trị, giá trị trung bình ~0.96, độ lệch chuẩn: ~0.23, Giá trị nhỏ nhất: 0.48,giá trị lớn nhất: 1.71 
- OD280: Không bị thiếu giá trị, giá trị trung bình ~2.61, độ lệch chuẩn: ~0.71, Giá trị nhỏ nhất: 1.27,giá trị lớn nhất: 4
- Proline: Không bị thiếu giá trị, giá trị trung bình ~746.89, độ lệch chuẩn: ~314.91, Giá trị nhỏ nhất: 278,giá trị lớn nhất: 1680

In [15]:
#Kiểm Tra độ tương quan
from pyspark.sql.functions import isnan, isnull, when, count, col
for col1 in numeric_cols:
    for col2 in numeric_cols:
        print("Chỉ số tướng quan giứa cột {} và cột {} là: {}".format(col1, col2, df.stat.corr(col1, col2)))

Chỉ số tướng quan giứa cột alcohol và cột alcohol là: 1.0
Chỉ số tướng quan giứa cột alcohol và cột Malic_Acid là: 0.09439693153136894
Chỉ số tướng quan giứa cột alcohol và cột Ash là: 0.21154460751527823
Chỉ số tướng quan giứa cột alcohol và cột Ash_Alcanity là: -0.31023516889213654
Chỉ số tướng quan giứa cột alcohol và cột Magnesium là: 0.27079827249167493
Chỉ số tướng quan giứa cột alcohol và cột Total_Phenols là: 0.2891011043777978
Chỉ số tướng quan giứa cột alcohol và cột Flavanoids là: 0.23681494465985697
Chỉ số tướng quan giứa cột alcohol và cột Nonflavanoid_Phenols là: -0.1559294638558501
Chỉ số tướng quan giứa cột alcohol và cột Proanthocyanins là: 0.1366979214547822
Chỉ số tướng quan giứa cột alcohol và cột Color_Intensity là: 0.5463641539473397
Chỉ số tướng quan giứa cột alcohol và cột Hue là: -0.07174720231913999
Chỉ số tướng quan giứa cột alcohol và cột OD280 là: 0.07234321627197132
Chỉ số tướng quan giứa cột alcohol và cột Proline là: 0.6437200499040044
Chỉ số tướng quan 

Hệ số tương quan là chỉ số thống kê đo lường mức độ mạnh yếu của mối quan hệ giữa hai biến số. Trong đó:

Hệ số tương quan có giá trị từ -1.0 đến 1.0. Kết quả được tính ra lớn hơn 1.0 hoặc nhỏ hơn -1 có nghĩa là có lỗi trong phép đo tương quan.

- Hệ số tương quan có giá trị âm cho thấy hai biến có mối quan hệ nghịch biến hoặc tương quan âm (nghịch biến tuyệt đối khi giá trị bằng -1)

- Hệ số tương quan có giá trị dương cho thấy mối quan hệ đồng biến hoặc tương quan dương (đồng biến tuyệt đối khi giá trị bằng 1)

- Tương quan bằng 0 cho hai biến độc lập với nhau.

### Trực quan dữ liệu

<b> Biểu đồ chỉ số tương quan các cột <b>

In [ ]:
numeric_data = df.select(numeric_cols).toPandas()
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
a = numeric_data.corr()
sns.heatmap(a, annot=True)

In [ ]:
axs = pd.plotting.scatter_matrix(numeric_data,figsize=(10,10))
n= len(numeric_cols)
for i in range(n):
    v = axs[i,0]
    v.yaxis.label.set_rotation(0)
    v.yaxis.label.set_ha('right')
    v.set_yticks(())
    h= axs[n-1,i]
    h.xaxis.label.set_rotation(90)
    h.set_xticks(())

<b> Biểu đồ thể hiện chỉ số của từng cột (giải thích số liệu ở phần mô tả dữ liệu) <b>

In [ ]:
d1=df.select('alcohol').toPandas()
d1.boxplot()

In [ ]:
d2=df.select('Malic_Acid').toPandas()
d2.boxplot()

In [ ]:
Ash=df.select('Ash').toPandas()
Ash.boxplot()

In [ ]:
Ash_Alcanity=df.select('Ash_Alcanity').toPandas()
Ash_Alcanity.boxplot()

In [ ]:
Magnesium=df.select('Magnesium').toPandas()
Magnesium.boxplot()

In [ ]:
Total_Phenols=df.select('Total_Phenols').toPandas()
Total_Phenols.boxplot()

In [ ]:
Flavanoids=df.select('Flavanoids').toPandas()
Flavanoids.boxplot()

In [ ]:
Nonflavanoid_Phenols=df.select('Nonflavanoid_Phenols').toPandas()
Nonflavanoid_Phenols.boxplot()

In [ ]:
Proanthocyanins=df.select('Proanthocyanins').toPandas()
Proanthocyanins.boxplot()

In [ ]:
Color_Intensity=df.select('Color_Intensity').toPandas()
Color_Intensity.boxplot()

In [ ]:
Hue=df.select('Hue').toPandas()
Hue.boxplot()

In [ ]:
OD280=df.select('OD280').toPandas()
OD280.boxplot()

In [ ]:
Proline=df.select('Proline').toPandas()
Proline.boxplot()

### Tìm và xử lý dữ liệu trống (Null)

In [ ]:
print("Số dòng null: {}".format(df.filter('alcohol is null').count()))
df.groupBy('alcohol').count().orderBy(col('count').desc()).show()

In [ ]:
print("Số dòng null: {}".format(df.filter('Malic_Acid is null').count()))
df.groupBy('Malic_Acid').count().orderBy(col('count').desc()).show()

In [ ]:
print("Số dòng null: {}".format(df.filter('Ash is null').count()))
df.groupBy('Ash').count().orderBy(col('count').desc()).show()

In [ ]:
print("Số dòng null: {}".format(df.filter('Ash_Alcanity is null').count()))
df.groupBy('Ash_Alcanity').count().orderBy(col('count').desc()).show()

In [ ]:
print("Số dòng null: {}".format(df.filter('Magnesium is null').count()))
df.groupBy('Magnesium').count().orderBy(col('count').desc()).show()

In [ ]:
print("Số dòng null: {}".format(df.filter('Total_Phenols is null').count()))
df.groupBy('Total_Phenols').count().orderBy(col('count').desc()).show()

In [ ]:
print("Số dòng null: {}".format(df.filter('Flavanoids is null').count()))
df.groupBy('Flavanoids').count().orderBy(col('count').desc()).show()

In [ ]:
print("Số dòng null: {}".format(df.filter('Nonflavanoid_Phenols is null').count()))
df.groupBy('Nonflavanoid_Phenols').count().orderBy(col('count').desc()).show()

In [ ]:
print("Số dòng null: {}".format(df.filter('Proanthocyanins is null').count()))
df.groupBy('Proanthocyanins').count().orderBy(col('count').desc()).show()

In [ ]:
print("Số dòng null: {}".format(df.filter('Color_Intensity is null').count()))
df.groupBy('Color_Intensity').count().orderBy(col('count').desc()).show()

In [ ]:
print("Số dòng null: {}".format(df.filter('Hue is null').count()))
df.groupBy('Hue').count().orderBy(col('count').desc()).show()

In [ ]:
print("Số dòng null: {}".format(df.filter('OD280 is null').count()))
df.groupBy('OD280').count().orderBy(col('count').desc()).show()

In [ ]:
print("Số dòng null: {}".format(df.filter('Proline is null').count()))
df.groupBy('Proline').count().orderBy(col('count').desc()).show()

Từ kết quả điếm số null ta nhận thấy dữ liệu khá hoàn chỉnh không có dữ liệu trống

# 3. Tiền xử lý & Xây dựng mô hình

### Tiền xử lý dữ liệu

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans

assembler = VectorAssembler(inputCols = df.columns, outputCol = 'features')
final_df = assembler.transform(df)
final_df.toPandas()

<b> StandardScaler: Chuẩn hóa dữ liệu <b>

In [ ]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol = 'features', outputCol = 'scaledFeatures')
scaler_model = scaler.fit(final_df)
final_df = scaler_model.transform(final_df)
final_df.toPandas()

In [ ]:
from pyspark.ml.feature import PCA
pca = PCA(k=2, inputCol = scaler.getOutputCol(), outputCol="pcaFeatures")

model = pca.fit(final_df)
transformed_feature = model.transform(final_df)
transformed_feature.show()

### Xây Dựng mô hình phân cụm

### Tìm số cụm K

<b> Sơ đồ phương pháp Elbow <b>

Phương pháp Elbow là một cách giúp ta lựa chọn được số lượng các cụm phù hợp dựa vào đồ thị trực quan hoá bằng cách nhìn vào sự suy giảm của hàm biến dạng và lựa chọn ra điểm khuỷ tay

In [ ]:
import numpy as np
cost = np.zeros(10)

for k in range(2,10):
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol('scaledFeatures')
    model = kmeans.fit(final_df)
    cost[k] = model.summary.trainingCost

# Plot the cost
df_cost = pd.DataFrame(cost[2:])
df_cost.columns = ["cost"]
new_col = [2,3,4,5,6,7,8, 9]
df_cost.insert(0, 'cluster', new_col)

import pylab as pl
pl.plot(df_cost.cluster, df_cost.cost, marker = 'o')
pl.xlabel('Number of Clusters')
pl.ylabel('Score')
pl.title('Elbow Curve')
pl.show()

Từ sơ đồ Elbow, ta nhận thấy số cụm K = 3 

<b> Sơ đồ sử dụng PCA <b>

Phép phân tích thành phần chính (Principal Components Analysis - PCA) là một thuật toán thống kê sử dụng phép biến đổi trực giao để biến đổi một tập hợp dữ liệu từ một không gian nhiều chiều sang một không gian mới ít chiều hơn (2 hoặc 3 chiều) nhằm tối ưu hóa việc thể hiện sự biến thiên của dữ liệ

In [ ]:
X_pca = transformed_feature.rdd.map(lambda row: row.pcaFeatures).collect()
X_pca = np.array(X_pca)
print(X_pca)

In [ ]:
df_pc = pd.DataFrame(X_pca, columns = ['PC1','PC2'])
df_pc

In [ ]:
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["figure.dpi"] = 80
plt.scatter(x=df_pc['PC1'], y=df_pc['PC2'], lw=2)
plt.xlabel("Principle Component 1")
plt.ylabel("Principle Component 2")
plt.title("Principle Components Analysis")
plt.show()

Có thể quan sát và dự đoán từ biểu đồ trên rằng dữ liệu có 3 cụm

### Chạy Kmean với số cụm K = 3 

Trong thuật toán K-means clustering, chúng ta không biết nhãn (label) của từng điểm dữ liệu. Mục đích là làm thể nào để phân dữ liệu thành các cụm (cluster) khác nhau sao cho dữ liệu trong cùng một cụm có tính chất giống nhau.

In [ ]:
kmeans = KMeans(featuresCol = 'scaledFeatures', k=3)
model = kmeans.fit(final_df)

<b> Xác định tâm cụm <b>

In [ ]:
centers = model.clusterCenters()
for center in centers:
    print(center)

<b> Phâm cụm dữ liệu <b>

In [ ]:
cluster = model.transform(final_df).select('scaledFeatures', 'prediction')
cluster.show(178)

In [ ]:
cluster.groupBy("prediction").count().show()

Ta nhận thấy:
- cụm thứ nhất (prediction = 0) có 64 dữ liệu có tính chất giống nhau
- cụm thứ hai (prediction = 1) có 63 dữ liệu có tính chất giống nhau
- cụm thứ ba (prediction = 2) có 51 dữ liệu có tính chất giống nhau

# 4. Đánh giá mô hình 

Để đánh giá chất lượng mô hình phân cụm ta có thể đánh giá thông qua một số phương pháp như sau:

- Khoảng cách / sự khác biệt giữa hai cụm bất kỳ phải lớn. (khoảng cách giữa các cụm): Giữa các cụm phải được tách biệt nhau hoàn toàn và sự khác biệt giữa 2 cụm phải đủ lớn để phân biệt 2 cụm với nhau.
- Chênh lệch giữa các điểm dữ liệu bên trong một cụm phải nhỏ. Chênh lệch ở đây thể hiện sự khác biệt với nhau về mặt tương đồng giữa 2 dữ liệu theo tiêu chí phân cụm.

silhouette là một số liệu được sử dụng để tính toán mức độ tốt của kỹ thuật phân cụm. Giá trị của nó nằm trong khoảng từ -1 đến 1.

1: Các cụm có nghĩa là cách xa nhau và phân biệt rõ ràng.

0: Các cụm có nghĩa là không quan tâm, hoặc chúng ta có thể nói rằng khoảng cách giữa các cụm là không đáng kể.

-1: Các cụm phương tiện được gán sai cách.

In [ ]:
pdt = model.transform(final_df)
from pyspark.ml.evaluation import ClusteringEvaluator
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(pdt)
print('Silhouette:', silhouette)

- Nhận thấy silhouette = 0.720945086354097 => cụm được chia tương đối rõ ràng
- Từ sơ đồ phân tàn ở phần phương pháp PCA ta nhận thấy: Dữ liệu đc phân tán tương đối gần nhau

=> từ 2 điều trên ta đưa ra được kết luận: <b> Mô hình hoạt động khá tốt <b>

In [ ]:
spark.stop()